In [2]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime 
import matplotlib.pyplot as plt
import re
from matplotlib.pyplot import figure

In [8]:
output_dir = "error_processed_data/"
file = "error_tree.pki"
raw_data = pd.read_pickle(output_dir + file)
raw_data = raw_data[["time", "User Name", "command", "full_command", "stderr"]]
raw_data

,time,User Name,command,full_command,stderr
0,2021-04-19 19:34:00,User_1,ls,ls,
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...
3,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:$HOME/bin,
4,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:test/bin,
...,...,...,...,...,...
341,2023-01-19 14:40:00,Lily,touch,touch ~/data-store/data/input/Discovery/TSVFil...,touch: cannot touch '/home/jovyan/data-store/d...
342,2022-12-15 19:11:00,Megan,clear,clear,
343,2022-12-15 19:11:00,Megan,ls,ls,
344,2022-12-15 19:14:00,Megan,cd,cd,


In [9]:
def error_splitter(errors):
    return errors.splitlines()
raw_data["error list"] = raw_data["stderr"].apply(lambda x: error_splitter(x))
raw_data

,time,User Name,command,full_command,stderr,error list
0,2021-04-19 19:34:00,User_1,ls,ls,,[]
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,[/home/supershell/.offline.sh: line 358: expor...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,[/home/supershell/.offline.sh: line 358: expor...
3,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:$HOME/bin,,[]
4,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:test/bin,,[]
...,...,...,...,...,...,...
341,2023-01-19 14:40:00,Lily,touch,touch ~/data-store/data/input/Discovery/TSVFil...,touch: cannot touch '/home/jovyan/data-store/d...,[touch: cannot touch '/home/jovyan/data-store/...
342,2022-12-15 19:11:00,Megan,clear,clear,,[]
343,2022-12-15 19:11:00,Megan,ls,ls,,[]
344,2022-12-15 19:14:00,Megan,cd,cd,,[]


In [10]:
expand_data = raw_data.explode('error list')
expand_data

,time,User Name,command,full_command,stderr,error list
0,2021-04-19 19:34:00,User_1,ls,ls,,NaN
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,/home/supershell/.offline.sh: line 358: export...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,/home/supershell/.offline.sh: line 358: export...
3,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:$HOME/bin,,NaN
4,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:test/bin,,NaN
...,...,...,...,...,...,...
341,2023-01-19 14:40:00,Lily,touch,touch ~/data-store/data/input/Discovery/TSVFil...,touch: cannot touch '/home/jovyan/data-store/d...,touch: cannot touch '/home/jovyan/data-store/d...
342,2022-12-15 19:11:00,Megan,clear,clear,,NaN
343,2022-12-15 19:11:00,Megan,ls,ls,,NaN
344,2022-12-15 19:14:00,Megan,cd,cd,,NaN


**Analyze on preceding commands of error**

In [11]:
file_name_replacer = re.compile(r'.+(\').+(\').+')
file_name = re.compile(r'(\').+(\')')

In [12]:
path_replacer = re.compile(r'.*(\/).+(: line )(\d+).*')
path_name = re.compile(r'(\/).+(: line )(\d+)')

In [13]:
name_between_colon = re.compile(r'.+(:).+(:).+')
colon_name = re.compile(r'(:).+(:)')

In [14]:
cd_replacer = re.compile(r'.*cd:(.+):.*')
cd_name = re.compile(r'cd:(.+):')

In [15]:
def replacer(error):
    if file_name_replacer.match(error):
        return re.sub(file_name, '$', error)
    elif path_replacer.match(error):
        error = re.sub(path_name, 'PATH', error)
        if file_name_replacer.match(error):
            return re.sub(file_name, '$', error)
        elif cd_replacer.match(error):
            return re.sub(cd_name, 'cd: $:', error)
        else:
            return error
    elif name_between_colon.match(error):
        return re.sub(colon_name, ': $:', error)

***Choose cat: cannot access $: No such file or directory as an example***

In [17]:
error_filter = (error_expand["no name"] == "cat: $: No such file or directory")

In [18]:
cat_error = error_expand[error_filter]
cat_error.head()

,time,User Name,command,full_command,stderr,error list,no name
68,2021-04-16 23:13:00,User_9,cat,"cat ""!!!!"" > goodbye.txt",cat: '!!!!': No such file or directory,[cat: '!!!!': No such file or directory],cat: $: No such file or directory
97,2021-04-17 13:50:00,User_9,cat,cat goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
172,2021-04-18 22:05:00,User_4,directory_grep_head_tail,NaN,cat: /home/jovyan/Bash/FilteredSortedOutput/Se...,[/home/jovyan/Bash/bin/directory_grep_head_tai...,cat: $: No such file or directory
176,2021-04-18 22:05:00,User_4,directory_grep_head_tail,NaN,cat: /home/jovyan/Bash/FilteredSortedOutput/Se...,[/home/jovyan/Bash/bin/directory_grep_head_tai...,cat: $: No such file or directory
410,2022-09-11 20:13:00,User_3,cat,cat hello.txt goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory


**Cat 1: cat "!!!">goodbye.txt**

In [19]:
cat_error_68 = error_expand.iloc[65:72]
cat_error_68

,time,User Name,command,full_command,stderr,error list,no name
65,2021-04-16 23:13:00,User_9,cat,cat goodbye.txt,,[],None
66,2021-04-16 23:13:00,User_9,cat,cat link_goodbye.txt,,[],None
67,2021-04-16 23:13:00,User_9,echo,"echo ""!!!!"" >> goodbye.txt",,[],None
68,2021-04-16 23:13:00,User_9,cat,"cat ""!!!!"" > goodbye.txt",cat: '!!!!': No such file or directory,[cat: '!!!!': No such file or directory],cat: $: No such file or directory
69,2021-04-16 23:15:00,User_9,cat,cat link_goodbye.txt,,[],None
70,2021-04-16 23:15:00,User_9,pico,NaN,,[],None
71,2021-04-16 23:15:00,User_9,diff,diff goodbye.txt copied_goodbye.txt,,[],None


**Cat 2: cat goodbye.txt**

In [20]:
cat_error_97 = error_expand.iloc[94:101]
cat_error_97

,time,User Name,command,full_command,stderr,error list,no name
94,2021-04-17 13:50:00,User_9,ls,ls -l,,[],None
95,2021-04-17 13:50:00,User_9,mv,mv goodbye.txt renamed_goodbye.txt,,[],None
96,2021-04-17 13:50:00,User_9,ls,ls -l,,[],None
97,2021-04-17 13:50:00,User_9,cat,cat goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
98,2021-04-17 13:57:00,User_9,ls,ls -r,,[],None
99,2021-04-17 13:57:00,User_9,ls,ls -l -r,,[],None
100,2021-04-17 13:57:00,User_9,ls,ls -R,,[],None


Error results because type the wrong file name

**Cat 3: cat hello.txt**

In [21]:
cat_error_410 = error_expand.iloc[407:414]
cat_error_410

,time,User Name,command,full_command,stderr,error list,no name
407,2022-09-11 20:12:00,User_3,pwd,pwd,,[],None
408,2022-09-11 20:13:00,User_3,cat,cat hello.txt goobye.txt,,[],None
409,2022-09-11 20:13:00,User_3,ls,ls,,[],None
410,2022-09-11 20:13:00,User_3,cat,cat hello.txt goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
411,2022-09-11 20:13:00,User_3,cat,cat hello.txt,,[],None
412,2022-09-11 20:20:00,User_3,cat,cat,,[],None
413,2022-09-11 20:22:00,User_3,ls,ls > ls.out,,[],None


Error results because type the wrong file name. a Typo

**Another Example: ls: cannot access $: No such file or directory**

In [22]:
error_filter_ls = (error_expand["no name"] == "ls: cannot access $: No such file or directory")

In [23]:
ls_error = error_expand[error_filter_ls]
ls_error.head(10)

,time,User Name,command,full_command,stderr,error list,no name
58,2021-04-16 22:56:00,User_9,ls,ls -l misch_child/goodbye.txt,ls: cannot access 'misch_child/goodbye.txt': N...,[ls: cannot access 'misch_child/goodbye.txt': ...,ls: cannot access $: No such file or directory
264,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,[ls: cannot access '/home/jovyan/RNA_SEQ_SAMPL...,ls: cannot access $: No such file or directory
270,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...,[ls: cannot access '/gunings/RNA_SEQ_SAMPLES':...,ls: cannot access $: No such file or directory
285,2022-09-11 18:33:00,User_3,ls,"ls stdout,txt.","ls: cannot access 'stdout,txt.': No such file ...","[ls: cannot access 'stdout,txt.': No such file...",ls: cannot access $: No such file or directory
286,2022-09-11 18:35:00,User_3,ls,ls RNAt,ls: cannot access 'RNAt': No such file or dire...,[ls: cannot access 'RNAt': No such file or dir...,ls: cannot access $: No such file or directory
287,2022-09-11 18:37:00,User_3,ls,ls RNA,ls: cannot access 'RNA': No such file or direc...,[ls: cannot access 'RNA': No such file or dire...,ls: cannot access $: No such file or directory
288,2022-09-11 18:37:00,User_3,ls,ls RNA^t,ls: cannot access 'RNA^t': No such file or dir...,[ls: cannot access 'RNA^t': No such file or di...,ls: cannot access $: No such file or directory
290,2022-09-11 18:51:00,User_3,ls,ls RNA^t,ls: cannot access 'RNA^t': No such file or dir...,[ls: cannot access 'RNA^t': No such file or di...,ls: cannot access $: No such file or directory
299,2022-09-11 18:57:00,User_3,ls,ls –l,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory
302,2022-09-11 19:17:00,User_3,ls,ls –l stdout.txt,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory


**ls example: 58**

In [24]:
ls_error_58 = error_expand.iloc[55:62]
ls_error_58

,time,User Name,command,full_command,stderr,error list,no name
55,2021-04-16 22:56:00,User_9,cd,cd ..,,[],None
56,2021-04-16 22:56:00,User_9,ls,ls,,[],None
57,2021-04-16 22:56:00,User_9,ln,ln -s goodbye.txt misc_child,,[],None
58,2021-04-16 22:56:00,User_9,ls,ls -l misch_child/goodbye.txt,ls: cannot access 'misch_child/goodbye.txt': N...,[ls: cannot access 'misch_child/goodbye.txt': ...,ls: cannot access $: No such file or directory
59,2021-04-16 22:56:00,User_9,ls,ls,,[],None
60,2021-04-16 22:57:00,User_9,ls,ls -l misc_child,,[],None
61,2021-04-16 23:05:00,User_9,cp,cp goodbye.txt copied_goodbye.txt,,[],None


Reason of error: type wrong path name

**ls example 264**

In [26]:
ls_error_264 = error_expand.iloc[261:270]
ls_error_264

,time,User Name,command,full_command,stderr,error list,no name
261,2022-03-02 20:57:00,User_8,ls,ls /home/jovyan/,,[],None
262,2022-03-02 20:58:00,User_8,ls,ls /home/jovyan/data,,[],None
263,2022-09-11 18:18:00,User_3,ls,ls /home/jovyan,,[],None
264,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,[ls: cannot access '/home/jovyan/RNA_SEQ_SAMPL...,ls: cannot access $: No such file or directory
265,2022-09-11 18:27:00,User_3,ls,ls ..,,[],None
266,2022-09-11 18:27:00,User_3,ls,ls,,[],None
267,2022-09-11 18:28:00,User_3,ls,ls,,[],None
268,2022-09-11 18:28:00,User_3,cd,cd home,,[],None
269,2022-09-11 18:28:00,User_3,ls,ls,,[],None


Reason of error: wrong path

**ls example 285**

In [27]:
ls_error_285 = error_expand.iloc[282:290]
ls_error_285

,time,User Name,command,full_command,stderr,error list,no name
282,2022-09-11 18:32:00,User_3,cd,cd ..,,[],None
283,2022-09-11 18:32:00,User_3,ls,ls,,[],None
284,2022-09-11 18:33:00,User_3,ls,ls stdout.txt,,[],None
285,2022-09-11 18:33:00,User_3,ls,"ls stdout,txt.","ls: cannot access 'stdout,txt.': No such file ...","[ls: cannot access 'stdout,txt.': No such file...",ls: cannot access $: No such file or directory
286,2022-09-11 18:35:00,User_3,ls,ls RNAt,ls: cannot access 'RNAt': No such file or dire...,[ls: cannot access 'RNAt': No such file or dir...,ls: cannot access $: No such file or directory
287,2022-09-11 18:37:00,User_3,ls,ls RNA,ls: cannot access 'RNA': No such file or direc...,[ls: cannot access 'RNA': No such file or dire...,ls: cannot access $: No such file or directory
288,2022-09-11 18:37:00,User_3,ls,ls RNA^t,ls: cannot access 'RNA^t': No such file or dir...,[ls: cannot access 'RNA^t': No such file or di...,ls: cannot access $: No such file or directory
289,2022-09-11 18:51:00,User_3,cd,cd home,,[],None


Reason of error: typo of file name. Should be period but use comma

**ls example 302**

In [28]:
ls_error_302 = error_expand.iloc[299:306]
ls_error_302

,time,User Name,command,full_command,stderr,error list,no name
299,2022-09-11 18:57:00,User_3,ls,ls –l,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory
300,2022-09-11 18:57:00,User_3,ls,ls -l,,[],None
301,2022-09-11 19:17:00,User_3,ls,ls -l stdout.txt,,[],None
302,2022-09-11 19:17:00,User_3,ls,ls –l stdout.txt,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory
303,2022-09-11 19:17:00,User_3,ls,ls,,[],None
304,2022-09-11 19:17:00,User_3,cd,cd ..,,[],None
305,2022-09-11 19:17:00,User_3,ls,ls,,[],None


Error reason: cannot link